In [3]:
import numpy as np
import cv2
import scipy
import matplotlib.pyplot as plt
import math
import os
from Utils.MiscUtils import *
from Utils.ImageUtils import *
from Utils.MathUtils import *
import scipy.optimize 
square_side = 12.5
# %load_ext autoreload
# %autoreload 2


In [4]:
def reprojectPointsAndGetError(A, kc, all_RT, all_image_corners, world_corners):

    error_mat = []
    alpha, gamma, beta, u0, v0, k1, k2 = extractParamFromA(A, kc)
    all_reprojected_points = []
    for i, image_corners in enumerate(all_image_corners): # for all images

        #RT for 3d world points
        RT = all_RT[i]
        #get ART for 2d world points
        RT3 = np.array([RT[:,0], RT[:,1], RT[:,3]]).reshape(3,3) #review
        RT3 = RT3.T
        ART3 = np.dot(A, RT3)

        image_total_error = 0
        reprojected_points = []
        for j in range(world_corners.shape[0]):

            world_point_2d = world_corners[j]
            world_point_2d_homo = np.array([world_point_2d[0], world_point_2d[1], 1]).reshape(3,1)
            world_point_3d_homo = np.array([world_point_2d[0], world_point_2d[1], 0, 1]).reshape(4,1)#cuz z = 0 on checkerboard

            #get radius of distortion
            XYZ = np.dot(RT, world_point_3d_homo)
            x =  XYZ[0] / XYZ[2]
            y = XYZ[1] / XYZ[2]
            # x = alpha * XYZ[0] / XYZ[2] #assume gamma as 0 
            # y = beta * XYZ[1] / XYZ[2] #assume gamma as 0
            r = np.sqrt(x**2 + y**2) #radius of distortion

            #observed image co-ordinates
            mij = image_corners[j]
            mij = np.array([mij[0], mij[1], 1], dtype = 'float').reshape(3,1)

            #projected image co-ordinates
            uvw = np.dot(ART3, world_point_2d_homo)
            u = uvw[0] / uvw[2]
            v = uvw[1] / uvw[2]

            u_dash = u + (u - u0) * (k1 * r**2 + k2 * r**4)
            v_dash = v + (v - v0) * (k1 * r**2 + k2 * r**4)
            reprojected_points.append([u_dash, v_dash])

            mij_dash = np.array([u_dash.item(), v_dash.item(), 1], dtype = 'float').reshape(3,1)

            #get error
            e = np.linalg.norm((mij - mij_dash), ord=2)
            image_total_error = image_total_error + e
        
        all_reprojected_points.append(reprojected_points)
        error_mat.append(image_total_error)
    error_mat = np.array(error_mat)
    error_average = np.sum(error_mat) / (len(all_image_corners) * world_corners.shape[0])
    # error_reprojection = np.sqrt(error_average)
    return error_average, all_reprojected_points


In [5]:
def lossFunc(x0, init_all_RT, all_image_corners, world_corners):

    A, kc = retrieveA(x0)
    alpha, gamma, beta, u0, v0, k1, k2 = x0

    error_mat = []

    for i, image_corners in enumerate(all_image_corners): # for all images

        #RT for 3d world points
        RT = init_all_RT[i]
        #get ART for 2d world points
        RT3 = np.array([RT[:,0], RT[:,1], RT[:,3]]).reshape(3,3) #review
        RT3 = RT3.T
        ART3 = np.dot(A, RT3)

        image_total_error = 0

        for j in range(world_corners.shape[0]):

            world_point_2d = world_corners[j]
            world_point_2d_homo = np.array([world_point_2d[0], world_point_2d[1], 1]).reshape(3,1)
            world_point_3d_homo = np.array([world_point_2d[0], world_point_2d[1], 0, 1]).reshape(4,1)

            #get radius of distortion
            XYZ = np.dot(RT, world_point_3d_homo)
            x =  XYZ[0] / XYZ[2]
            y = XYZ[1] / XYZ[2]
            # x = alpha * XYZ[0] / XYZ[2] #assume gamma as 0 
            # y = beta * XYZ[1] / XYZ[2] #assume gamma as 0
            r = np.sqrt(x**2 + y**2) #radius of distortion

            #observed image co-ordinates
            mij = image_corners[j]
            mij = np.array([mij[0], mij[1], 1], dtype = 'float').reshape(3,1)

            #projected image co-ordinates
            uvw = np.dot(ART3, world_point_2d_homo)
            u = uvw[0] / uvw[2]
            v = uvw[1] / uvw[2]

            u_dash = u + (u - u0) * (k1 * r**2 + k2 * r**4)
            v_dash = v + (v - v0) * (k1 * r**2 + k2 * r**4)

            mij_dash = np.array([u_dash.item(), v_dash.item(), 1], dtype = 'float').reshape(3,1)

            #get error
            e = np.linalg.norm((mij - mij_dash), ord=2)
            image_total_error = image_total_error + e

        error_mat.append(image_total_error / 54)
    
    return np.array(error_mat)
        


In [8]:
def run(mode):

    if (mode == "ir"):
        folder_name = r'C:\Users\Asus\Documents\Shuna Ni\images\ir_mod'
        save_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\ir_saved"  
    elif (mode=="rgb"):
        folder_name = r'C:\Users\Asus\Documents\Shuna Ni\images\rgb_mod'
        save_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\rgb_saved"

    images = loadImages(folder_name)
    h, w = [6,7]
    all_image_corners = getImagesPoints(images, h, w)
    print(len(all_image_corners) , len(all_image_corners[0]))
    world_corners = getWorldPoints(square_side, h, w)

    displayCorners(images, all_image_corners, h, w, save_folder)

    print("Calculating H for %d images", len(images))
    all_H_init = getAllH(all_image_corners, square_side, h, w)
    print("Calculating B")
    B_init = getB(all_H_init)
    print("Estimated B = ", B_init)
    print("Calculating A")
    A_init = getA(B_init)
    print("Initialized A = ",A_init)
    print("Calculating rotation and translation")
    all_RT_init = getRotationAndTrans(A_init, all_H_init)
    print("Init Kc")
    kc_init = np.array([0,0]).reshape(2,1)
    print("Initialized kc = ", kc_init)


    return A_init, kc_init, all_RT_init, all_image_corners, all_H_init, world_corners

    # print("Optimizing ...")
    # x0 = extractParamFromA(A_init, kc_init)
    # res = scipy.optimize.least_squares(fun=lossFunc, x0=x0, method="lm", args=[all_RT_init, all_image_corners, world_corners])
    # x1 = res.x
    # A_new, kc_new = retrieveA(x1)

    # # A_new = AK[0]
    # # kc_new = AK[1]
    
    # previous_error, _ = reprojectPointsAndGetError(A_init, kc_init, all_RT_init, all_image_corners, world_corners)
    # att_RT_new = getRotationAndTrans(A_new, all_H_init)
    # new_error, points = reprojectPointsAndGetError(A_new, kc_new, att_RT_new, all_image_corners, world_corners)

    # print("The error befor optimization was ", previous_error)
    # print("The error after optimization is ", new_error)
    # print("The A matrix is: ", A_new)

    # K = np.array(A_new, np.float32).reshape(3,3)
    # D = np.array([kc_new[0][0],kc_new[1][0], 0, 0] , np.float32)
    # for i,image_points in enumerate(points):
    #     image = cv2.undistort(images[i], K, D)
    #     for point in image_points:
    #         x = int(point[0].item())
    #         y = int(point[1].item())
    #         image = cv2.circle(image, (x, y), 5, (0, 0, 255), 3)
    #     # cv2.imshow('frame', image)
    #     filename = save_folder + str(i) + "reproj.png"
    #     cv2.imwrite(filename, image)
    #     # cv2.waitKey()

    # cv2.destroyAllWindows()
    # return A_new, kc_new, att_RT_new, all_image_corners, all_H_init, world_corners
    # return(A_new,kc_new,att_RT_new,all_RT_init)


In [9]:
#run once

rgb_A, rgb_kc, rgb_rt, rgb_corners, rgb_h, wc = run("rgb")
ir_A, ir_kc, ir_rt, ir_corners, ir_h, wc_1 = run("ir")

Loading images from  C:\Users\Asus\Documents\Shuna Ni\images\rgb_mod
7
7 42
Calculating H for %d images 7
Calculating B
B matrix is  [ 2.39542221e-06 -2.24840500e-08  2.42254636e-06 -7.56087644e-04
 -6.04368049e-04  9.99999532e-01]
Estimated B =  [[ 2.39542221e-06 -2.24840500e-08 -7.56087644e-04]
 [-2.24840500e-08  2.42254636e-06 -6.04368049e-04]
 [-7.56087644e-04 -6.04368049e-04  9.99999532e-01]]
Calculating A
Initialized A =  [[503.3878367    4.69860669 318.00792333]
 [  0.         500.58360707 252.42784461]
 [  0.           0.           1.        ]]
Calculating rotation and translation
Init Kc
Initialized kc =  [[0]
 [0]]
Loading images from  C:\Users\Asus\Documents\Shuna Ni\images\ir_mod
7
7 42
Calculating H for %d images 7
Calculating B
B matrix is  [ 1.08070634e-06  4.91639657e-08  1.16190843e-06 -4.02615985e-04
 -4.41719989e-04  9.99999821e-01]
Estimated B =  [[ 1.08070634e-06  4.91639657e-08 -4.02615985e-04]
 [ 4.91639657e-08  1.16190843e-06 -4.41719989e-04]
 [-4.02615985e-04 -

In [10]:
rgb_to_ir = []
for i in range(len(rgb_h)):
    H_rgb_inv = np.linalg.inv(rgb_h[i])
    H_ir = ir_h[i]
    rgb_to_ir.append(np.matmul(H_ir, H_rgb_inv))

# print(H_inv.shape, rgb_corners[0][0])
error_by_image = []
all_projections = []
for i in range(len(rgb_corners)):
    projections = []
    error = 0
    for j in range(len(rgb_corners[i])):
        rgb_corner = rgb_corners[i][j]
        corner2 = np.array([rgb_corner[0],rgb_corner[1],1]).reshape(3,1)
        calculated_point_3 = np.matmul(rgb_to_ir[i],corner2)
        calculated_point_2 = np.array([calculated_point_3[0][0]/calculated_point_3[2][0],calculated_point_3[1][0]/calculated_point_3[2][0] ])
        projections.append(calculated_point_2)
        actual_detected_corner = ir_corners[i][j]
        
        error += np.linalg.norm(calculated_point_2 - actual_detected_corner)
    all_projections.append(projections)
    mean_error = error/j
    error_by_image.append(mean_error) 

print("Re-projection error for each image", error_by_image)
print("Mean error", sum(error_by_image)/len(error_by_image))

Re-projection error for each image [0.697342344273477, 0.5814622574160199, 0.9821334309978444, 1.0366495552007313, 0.8302213141083916, 0.6632218343292802, 0.7428255508159852]
Mean error 0.7905508981631043


In [11]:
open_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\ir_mod"
save_folder = r"C:\Users\Asus\Documents\Shuna Ni\images\reproj_ir"
images = loadImages(open_folder)
for i,image in enumerate(images):
    drawn = cv2.drawChessboardCorners(image, (7, 6), ir_corners[i], True)
    # cv2.imshow('Chessboard',drawn)
    # cv2.waitKey(0)
    for point in all_projections[i]:
        x = int(point[0])
        y = int(point[1])
        image = cv2.circle(drawn, (x, y), 5, (0, 0, 255), 1)
        
    filename = save_folder + "\\" + str(i) + "reproj.png"
    cv2.imwrite(filename, image)

Loading images from  C:\Users\Asus\Documents\Shuna Ni\images\ir_mod
